In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [2]:
import logging
import delta_sharing
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import general_functions.databricks_client as db_client
from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import call_api_with_accountId, send_to_innkeepr_api_paginated

In [3]:
url = return_api_url()
account_id = return_account_ids()

In [4]:
profile_path = db_client.return_databricks_client()
table_path = f"{profile_path}#delta_share_events.monitoring.datashifts"
df = delta_sharing.load_as_pandas(table_path)#, limit=1000)

In [5]:
df.to_csv("DataChecks/datashift/check_datashift_data.csv", index=False)

In [6]:
df.head()

In [7]:
df["workspace"] = df["workspace_id"].apply(lambda x: [acc["name"] for acc in account_id if acc["id"] == x])
df["workspace"] = df["workspace"].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
df.head()

In [8]:
df.to_csv("DataChecks/datashift/check_datashift_data.csv", index=False)

In [9]:
df.groupby("workspace")["mean_count_sessions"].mean().round(2).sort_values(ascending=False)

In [10]:
df.groupby("workspace")["mean_count_sessions"].describe()#.sort_values(ascending=False)

In [11]:
df.groupby("workspace")["mean_count_anonymousIds"].mean().sort_values(ascending=False)

# Plot per customer

In [12]:
customer = "Tchibo"
min_date = pd.to_datetime(df["created"].max()) - timedelta(days=20)
min_date = min_date.strftime("%Y-%m-%d")
print(f"Filtering data for customer: {customer} since {min_date}")
df_customer = df[df["workspace"] == customer]
df_customer["created"] = pd.to_datetime(df_customer["created"])
df_customer

In [13]:
sns.lineplot(
    data=df_customer[df_customer["created"]>=min_date],
    x="created",
    y="mean_count_sessions",
    marker="o",
    label=customer
)
sns.lineplot(
    data=df_customer[df_customer["created"]>=min_date],
    x="created",
    y="count_sessions_max_date",
    marker="o",
    label=customer
)
plt.xticks(rotation=45)
plt.title("Mean Count Sessions over Time for " + customer)
plt.grid(True)
plt.ylim(0, df_customer["count_sessions_max_date"].max() + 20)


In [14]:
sns.lineplot(
    data=df_customer[df_customer["created"]>=min_date],
    x="created",
    y="mean_count_anonymousIds",
    marker="o",
    hue="workspace"
)
sns.lineplot(
    data=df_customer[df_customer["created"]>=min_date],
    x="created",
    y="count_anonymouIds_max_date",
    marker="x",
    hue="workspace",
    color="o"
)
plt.xticks(rotation=45)
plt.title("Mean Count AnonymousIds per Account over Time")
plt.grid(True)
plt.ylim(0, df_customer["count_anonymouIds_max_date"].max() + 10)